In [1]:
import cobra
import copy
import numpy as np
import pandas as pd 

In [2]:
#Add the (linear) constraint for the allocation of enzymatic resources based on kapps (1/h), MW (g/mmol), and total proteome to use for the constrained reactions (in g/gdW)
def add_molecular_crowding_constraint(m,kapp_rxns,dir,kapp,mw,tot_protein):
    coefficients=dict()
    for i in range(len(kapp_rxns)):
        cur_kapp_rxn=kapp_rxns[i]
        cur=m.reactions.get_by_id(cur_kapp_rxn)
        if dir[i]=='fwd':
            coefficients[cur.forward_variable] = mw[i]/kapp[i]
        elif dir[i]=='bwd':
            coefficients[cur.reverse_variable] = mw[i]/kapp[i]
    mc_constraint = m.problem.Constraint(0,lb=0,ub=tot_protein)
    m.add_cons_vars(mc_constraint)
    m.solver.update()
    mc_constraint.set_linear_coefficients(coefficients=coefficients)

In [3]:
#Load the Reduced Model
model=cobra.io.load_json_model('../Model/redmod.json')

In [4]:
#Read the Molecular Crowding data and store it in various lists
kapp_tbl=pd.read_csv('kapp_w_mass.csv',index_col='ix')

kapp_rxns=kapp_tbl['id'].to_list() #List of Reaction that will appear in the MC constraint
kapp_dir=kapp_tbl['dir'].to_list() #Direction in which the constraint is being applied ('fwd' or 'bwd')
kapp=np.array(kapp_tbl['kapp'].to_list())*3600 #kapps, scaled so that they are expressed in (1/h)
kapp=kapp.tolist()
mw=np.array(kapp_tbl['pp_mass'].to_list())/1000 #MWs, scaled so that they are expressed in g/mmol
mw=mw.tolist()
tot_protein=np.sum(kapp_tbl['pp_abundance mass'].to_list()) #Total proteome available, based on protein abundance measurements

print('Total Proteome available: %f g/gDW'%tot_protein)

Total Proteome available: 0.108584 g/gDW


In [5]:
model.reactions.get_by_id('EX_glc__D_e').lower_bound=-1000 #remove original glucose bound
add_molecular_crowding_constraint(model,kapp_rxns,kapp_dir,kapp,mw,tot_protein)

In [6]:

#1. pFBA (further minimize the norm of the flux vector across the optimal subspace)
pfba_solution = cobra.flux_analysis.pfba(model)
print('pFBA Objective Value:  '+ str(pfba_solution.fluxes['RBIO']))
pfba_solution.fluxes.to_csv('output/pfba_results.csv')

pFBA Objective Value:  1.0124947875270953


In [7]:
#2. Flux Variability Analysis
fva_solution=cobra.flux_analysis.flux_variability_analysis(model,fraction_of_optimum= 1)
fva_solution.to_csv('output/fva_solution.csv')